In [1]:
import requests,bs4,string,csv,traceback,os
import pandas as pd
import numpy as np
import re 
from datetime import datetime, timedelta
import logging
import logging.config
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

def scrape_financial_times():
    config=pd.read_csv('../Config/config.cfg',sep='=',index_col=0)
    logging.config.fileConfig(fname='../Config/logging.conf', disable_existing_loggers=False)
    lastExtractDate=datetime.strptime(config.Value['FinancialTimes_LastDatePulled'],"%d/%m/%Y")
    logger=logging.getLogger('scrape_financial_times')
    logger.info('Started Scrapping the data')
    data_file=config.Value['NewsDataDir']+'/financial_times.csv'
    csv.register_dialect('myDialect',delimiter = '|',escapechar='\\',lineterminator = '\n',quoting=csv.QUOTE_NONE,skipinitialspace=True)
    pageno=1
    condition=True
    while condition:
        if pageno==1:
            weblink='https://www.ft.com/oil'
        else:
            weblink='https://www.ft.com/oil?page='+str(pageno)
        fhand=requests.get(weblink)
#         print(fhand.read())
        if (fhand.text=='Maximum page number exceeded'):
            break
        soup=bs4.BeautifulSoup(fhand.text,'html.parser')
        li_lst=soup.findAll('li',{'class':'o-teaser-collection__item o-grid-row'})
        if li_lst is None:
            li_lst=soup.findAll('li',{'class':'o-teaser-collection__item o-grid-row '})
        else:
            li_lst.extend(soup.findAll('li',{'class':'o-teaser-collection__item o-grid-row '}))
        with open(data_file, 'a') as f:
#             logger.info('Opened file '+data_file)
            writer = csv.writer(f, dialect='myDialect')
            for li in li_lst:
                try:
                    if li and li.find('time',{'data-o-component':'o-date'}):
                        date_object = datetime.strptime(li.find('time',{'data-o-component':'o-date'}).text, "%A, %d %B, %Y")
                        if date_object<=lastExtractDate:
                            condition=False
                            break
                        writer.writerow([date_object.strftime("%d/%m/%Y"),li.find('div',{'class':'o-teaser__heading'}).text.encode('utf8','ignore').decode('utf-8', 'replace')])
                except AttributeError as ae:
                    logger.exception('Error:'+str(ae)+'|Link:'+weblink+'Tag:'+str(li))
                except UnicodeEncodeError as ue:
                    logger.exception('Error:'+str(ue)+'|Link:'+weblink+'Tag:'+str(li))
                except:
                    logger.exception('Error:|Link:'+weblink+'Tag:'+str(li))
        f.close()
        fhand.close()
        logger.info('Scrapping Completed for '+weblink)
        pageno=pageno+1
    config.Value['FinancialTimes_LastDatePulled']=datetime.strftime(datetime.now()- timedelta(1),"%d/%m/%Y")
    config.to_csv('../Config/config.cfg',sep='=')
    logger.info('FinancialTimes:completed Successfully')

def scrape_oilprice():
    config=pd.read_csv('../Config/config.cfg',sep='=',index_col=0)
    logging.config.fileConfig(fname='../Config/logging.conf', disable_existing_loggers=False)
    lastExtractDate=datetime.strptime(config.Value['OilPrice_LastDatePulled'],"%d/%m/%Y")
    logger=logging.getLogger('scrape_oilprice')
    logger.info('Started Scrapping the data')
    data_file=config.Value['NewsDataDir']+'/oilprice.csv'
    csv.register_dialect('myDialect',delimiter = '|',escapechar='\\',lineterminator = '\n',quoting=csv.QUOTE_NONE,skipinitialspace=True)
    pageno=1
    condition=True
    while condition:
        if pageno==1:
            weblink='https://oilprice.com/Energy/Crude-Oil/'
        else:
            weblink='https://oilprice.com/Energy/Crude-Oil/Page-'+str(pageno)+".html"
        fhand=requests.get(weblink)
        soup=bs4.BeautifulSoup(fhand.text,'html.parser')
        article_lst=soup.findAll('div',{'class':'categoryArticle__content'})
        with open(data_file, 'a') as f:
#             logger.info('Opened file '+data_file)
            writer = csv.writer(f, dialect='myDialect')
            for article in article_lst:
                try:
                    if article:
                        date_object = datetime.strptime(article.find('p',{'class':'categoryArticle__meta'}).text.split('at')[0].strip(), "%b %d, %Y")
                        if date_object<=lastExtractDate:
                            condition=False
                            break
                        writer.writerow([date_object.strftime("%d/%m/%Y"),article.find('h2',{'class':'categoryArticle__title'}).text.strip().encode('utf8','ignore').decode('utf-8', 'replace')])
                except AttributeError as ae:
                    logger.exception('Error:'+str(ae)+'|Link:'+weblink+'Tag:'+str(li))
                except UnicodeEncodeError as ue:
                    logger.exception('Error:'+str(ue)+'|Link:'+weblink+'Tag:'+str(li))
                except:
                    logger.exception('Error:|Link:'+weblink+'Tag:'+str(li))
        f.close()
        fhand.close()
        logger.info('Scrapping Completed for '+weblink)
        pageno=pageno+1
        max_page_no=int(soup.find('span',{'class':'num_pages'}).text.strip('/'))
        if (pageno>max_page_no):
            break
    config.Value['OilPrice_LastDatePulled']=datetime.strftime(datetime.now()- timedelta(1),"%d/%m/%Y")
    config.to_csv('../Config/config.cfg',sep='=')
    logger.info('OilPrice:completed Successfully')
    
def scrape_moneycontrol():
    config=pd.read_csv('../Config/config.cfg',sep='=',index_col=0)
    logging.config.fileConfig(fname='../Config/logging.conf', disable_existing_loggers=False)
    lastExtractDate=datetime.strptime(config.Value['MoneyControl_LastDatePulled'],"%d/%m/%Y")
    logger=logging.getLogger('scrape_moneycontrol')
    logger.info('Started Scrapping the data')
    data_file=config.Value['NewsDataDir']+'/moneycontrol.csv'
    csv.register_dialect('myDialect',delimiter = '|',escapechar='\\',lineterminator = '\n',quoting=csv.QUOTE_NONE,skipinitialspace=True)
    pageno=1
    condition=True
    while condition:
        if pageno==1:
            weblink='https://www.moneycontrol.com/news/commodities-news-94.html/'
        else:
            weblink='https://www.moneycontrol.com/news/commodities-news-94.html/page-'+str(pageno)+'/'
        fhand=requests.get(weblink)
        soup=bs4.BeautifulSoup(fhand.text,'html.parser')
        article_lst=soup.find('ul', {'id': 'cagetory'}).find_all('li', {'id':re.compile('newslist-\d+')})
        with open(data_file, 'a') as f:
#             logger.info('Opened file '+data_file)
            writer = csv.writer(f, dialect='myDialect')
            for article in article_lst:
                try:
                    if article:
                        if re.search('oil|us|opec|saudi|crude|energy|shale|gas|dollar|iran|could|stocks|dollarbn|premium|\
                        russia|week|arabia|bp|supply|china|sea|aramco|rise|peak|shell|trump|pipeline|uk|sanctions|brent|\
                        venezuela|investors|growth|producers|iea|nigeria|drilling|gulf|rosneft|russian|saudis|barrel|iraq\
                        |fracking|bpd|glut|fuel|mexico|eia|chevron|chinas|libya|opecs|iranian|permian|canadian|chinese|\
                        reserves|arctic|petrobras|gasoline|russias|texas|oils|tullow|lng|surge|venezuelas|canadas|canada\
                        |barrels|oilfield|mexicos|irans|asian|natural|carbon|arabias|norway|drillers|european|fossil|\
                        libyan|gazprom|fuels|iraqi|refiners|norways|petroleum|tankers|nigerias|libyas|wti|venezuelan|\
                        emissions|repsol|statoil|brazil|mexican|electric|refinery|kurdistan|nigerian|australian|refining\
                        |petrol|shows|iraqs|kuwait|syria|kurdish|kazakhstan|coal|turkey|refineries|uae|petrofac|lukoil'\
                                     ,article.find('a').get('title').lower()):
                            date_object = datetime.strptime(' '.join(article.find('span').text.split()[:3]).strip(), "%B %d, %Y")
                            if date_object<=lastExtractDate:
                                condition=False
                                break
                            writer.writerow([date_object.strftime("%d/%m/%Y"),article.find('a').get('title').strip().encode('utf8','ignore').decode('utf-8', 'replace')])
                        else:
                            logger.debug('No Crude oil news found')
                except AttributeError as ae:
                    logger.exception('Error:'+str(ae)+'|Link:'+weblink+'Tag:'+str(li))
                except UnicodeEncodeError as ue:
                    logger.exception('Error:'+str(ue)+'|Link:'+weblink+'Tag:'+str(li))
                except:
                    logger.exception('Error:|Link:'+weblink+'Tag:'+str(li))
        f.close()
        fhand.close()
        logger.info('Scrapping Completed for '+weblink)
        pageno=pageno+1
        for last_anchor in soup.find('div', {'class': 'pagenation'}).find_all('a', {'class':'last'}):
            if last_anchor.text.strip(' »')=='Last':
                max_page_no=int(last_anchor['data-page'])
                break
        if (pageno>max_page_no):
            break
    config.Value['MoneyControl_LastDatePulled']=datetime.strftime(datetime.now()- timedelta(1),"%d/%m/%Y")
    config.to_csv('../Config/config.cfg',sep='=')
    logger.info('MoneyControl:completed Successfully')
    
def scrape_CrudePrice():
    config=pd.read_csv('../Config/config.cfg',sep='=',index_col=0)
    logging.config.fileConfig(fname='../Config/logging.conf', disable_existing_loggers=False)
    lastExtractDate=datetime.strptime(config.Value['CrudePrice_LastDatePulled'],"%d/%m/%Y")
    logger=logging.getLogger('scrape_CrudePrice')
    logger.info('Started Scrapping the data')
    data_file=config.Value['NewsDataDir']+'/CrudePrice.csv'
    csv.register_dialect('myDialect',delimiter = '|',escapechar='\\',lineterminator = '\n',quoting=csv.QUOTE_NONE,skipinitialspace=True)

    weblink='https://markets.businessinsider.com/commodities/historical-prices/oil-price/usd/'+lastExtractDate.strftime("%d.%m.%Y")+'_'+datetime.now().strftime("%d.%m.%Y")+'?type=wti'
#     print(weblink)

    option = webdriver.ChromeOptions()
    option.add_argument("—incognito")
    browser = webdriver.Chrome(executable_path='./chromedriver')
    browser.get(weblink)
    browser.implicitly_wait(30)
    tr_lst=browser.find_elements_by_xpath("//div[@class='table-responsive']/table/tbody/tr")
    with open(data_file, 'a') as f:
        writer = csv.writer(f, dialect='myDialect')
        for tr in tr_lst:
            td_lst=tr.find_elements_by_tag_name('td')
            rec=[]
            for td in td_lst:
                rec.append(td.text)
            if len(rec)>1:
                writer.writerow(rec)
    f.close()
    browser.close()
    logger.info('Scrapping Completed for '+weblink)
    config.Value['CrudePrice_LastDatePulled']=datetime.strftime(datetime.now()- timedelta(1),"%d/%m/%Y")
    config.to_csv('../Config/config.cfg',sep='=')
    logger.info('CrudePrice:completed Successfully')
def main():
    try:
        scrape_financial_times()
    except:
        logging.exception('Error While scriping Financial Times')
    try:
        scrape_oilprice()
    except:
        logging.exception('Error While scriping Oil price')
    try:
        scrape_moneycontrol()
    except:
        logging.exception('Error While scriping MoneyControl')
#     try:
#         scrape_CrudePrice()
#     except:
#         logging.exception('Error While scriping Crude Oil Price')
logging.shutdown()

In [114]:
main()

--- Logging error ---
Traceback (most recent call last):
  File "<ipython-input-113-9e2d70a1d3c0>", line 49, in scrape_financial_times
    writer.writerow([date_object.strftime("%d/%m/%Y"),li.find('div',{'class':'o-teaser__heading'}).text.encode('utf8','ignore').decode('utf-8', 'replace')])
  File "C:\ProgramData\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u2009' in position 41: character maps to <undefined>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 1037, in emit
    stream.write(msg + self.terminator)
  File "C:\ProgramData\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u2009' in posi

--- Logging error ---
Traceback (most recent call last):
  File "<ipython-input-113-9e2d70a1d3c0>", line 49, in scrape_financial_times
    writer.writerow([date_object.strftime("%d/%m/%Y"),li.find('div',{'class':'o-teaser__heading'}).text.encode('utf8','ignore').decode('utf-8', 'replace')])
  File "C:\ProgramData\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u2009' in position 51: character maps to <undefined>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 1037, in emit
    stream.write(msg + self.terminator)
  File "C:\ProgramData\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u2009' in posi

--- Logging error ---
Traceback (most recent call last):
  File "<ipython-input-113-9e2d70a1d3c0>", line 49, in scrape_financial_times
    writer.writerow([date_object.strftime("%d/%m/%Y"),li.find('div',{'class':'o-teaser__heading'}).text.encode('utf8','ignore').decode('utf-8', 'replace')])
  File "C:\ProgramData\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u200a' in position 38: character maps to <undefined>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 1037, in emit
    stream.write(msg + self.terminator)
  File "C:\ProgramData\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u200a' in posi

--- Logging error ---
Traceback (most recent call last):
  File "<ipython-input-113-9e2d70a1d3c0>", line 49, in scrape_financial_times
    writer.writerow([date_object.strftime("%d/%m/%Y"),li.find('div',{'class':'o-teaser__heading'}).text.encode('utf8','ignore').decode('utf-8', 'replace')])
  File "C:\ProgramData\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u2009' in position 18: character maps to <undefined>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 1037, in emit
    stream.write(msg + self.terminator)
  File "C:\ProgramData\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u2009' in posi

--- Logging error ---
Traceback (most recent call last):
  File "<ipython-input-113-9e2d70a1d3c0>", line 49, in scrape_financial_times
    writer.writerow([date_object.strftime("%d/%m/%Y"),li.find('div',{'class':'o-teaser__heading'}).text.encode('utf8','ignore').decode('utf-8', 'replace')])
  File "C:\ProgramData\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u2032' in position 15: character maps to <undefined>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\__init__.py", line 1037, in emit
    stream.write(msg + self.terminator)
  File "C:\ProgramData\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u2032' in posi

In [100]:
def scrape_Investing(): 
    config=pd.read_csv('../Config/config.cfg',sep='=',index_col=0)
    logging.config.fileConfig(fname='../Config/logging.conf', disable_existing_loggers=False)
    lastExtractDate=datetime.strptime(config.Value['Investing_LastDatePulled'],"%d/%m/%Y")
    logger=logging.getLogger('scrape_Investing')
    logger.info('Started Scrapping the data')
    data_file=config.Value['NewsDataDir']+'/investing.csv'
    csv.register_dialect('myDialect',delimiter = '|',escapechar='\\',lineterminator = '\n',quoting=csv.QUOTE_NONE,skipinitialspace=True)
    
    weblink='https://www.investing.com/news/commodities-news/'
    option = webdriver.ChromeOptions()
    option.add_argument("—incognito")
    # option.add_argument("headless")
    browser = webdriver.Chrome(executable_path='./chromedriver',options=option)
    browser.get(weblink)
#     browser.maximize_window()
    browser.implicitly_wait(np.random.randint(5,20))
    
    # promo_cross=browser.find_elements_by_xpath("//div[@id='PromoteSignUpPopUp']/div[@class='right']/i")[0]
    # promo_cross.click()
    flag='Running'
    PageNo=1844
    
    while flag!='Done':
        with open(data_file, 'a') as f:
            browser.get(weblink+str(PageNo))
#     browser.maximize_window()
            browser.implicitly_wait(np.random.randint(5,20))
            writer = csv.writer(f, dialect='myDialect')
            div=browser.find_elements_by_xpath("//div[@class='wrapper']/section[@id='leftColumn']/div[@class='largeTitle']/article/div[@class='textDiv']")
            for dv in div:
                news=dv.find_elements_by_tag_name('a')[0].text.strip()
                if re.search('oil|us|opec|shale|crude|saudi|peak|iran|dollar|gas|energy|china|arabia|russia|bpd|rig|offshore|eia|draw|boost',news.lower()):
                    time=dv.find_elements_by_xpath('//*[@class="articleDetails"]/span[@class="date"]')[0].text
                    time=time.lower().replace('-','').strip()
                    if re.search('hour|minute|now',time):
                        time=datetime.now()
                    else:
                        time=datetime.strptime(time, "%b %d, %Y")
                    if time<=lastExtractDate:
                        flag='Done'
                        break
                    if len(news)!=0:
                        writer.writerow([time.strftime("%d/%m/%Y"),news])     
            try:
                nextButton=browser.find_elements_by_xpath('//div[@class="sideDiv inlineblock text_align_lang_base_2"]/a')[0]
                nextButton.click()
            except:
                flag='Done'
                browser.implicitly_wait(np.random.randint(5,20))
            logger.info('Scrapping Completed for '+weblink)
    f.close()
    browser.close()
    logger.info('Scrapping Completed for '+weblink)
    config.Value['Investing_LastDatePulled']=datetime.strftime(datetime.now()- timedelta(1),"%d/%m/%Y")
    config.to_csv('../Config/config.cfg',sep='=')
    logger.info('Investing:completed Successfully')

In [101]:
scrape_Investing()

In [8]:
config=pd.read_csv('../Config/config.cfg',sep='=',index_col=0)
data_file=config.Value['NewsDataDir']+'/investing.csv'
csv.register_dialect('myDialect',delimiter = '|',escapechar='\\',lineterminator = '\n',quoting=csv.QUOTE_NONE,skipinitialspace=True)
    
weblink='https://www.investing.com/news/commodities-news/'
option = webdriver.ChromeOptions()
option.add_argument("—incognito")
    # option.add_argument("headless")
browser = webdriver.Chrome(executable_path='./chromedriver',options=option)

PageNo=1865
    
with open(data_file, 'a') as f:
    writer = csv.writer(f, dialect='myDialect')
    while PageNo!=0:
        browser.get(weblink+str(PageNo))
#     browser.maximize_window()
#         browser.implicitly_wait(np.random.randint(5,20))
        div=browser.find_elements_by_xpath("//div[@class='wrapper']/section[@id='leftColumn']/div[@class='largeTitle']/article/div[@class='textDiv']")
        for dv in div:
            news=dv.find_elements_by_tag_name('a')[0].text.strip()
            if re.search('oil|us|opec|saudi|crude|energy|shale|gas|dollar|iran|could|stocks|dollarbn|premium|russia|week|\
            arabia|bp|supply|china|sea|aramco|rise|peak|shell|trump|pipeline|uk|sanctions|brent|venezuela|investors|growth\
            |producers|iea|nigeria|drilling|gulf|rosneft|russian|saudis|barrel|iraq|fracking|bpd|glut|fuel|mexico|eia|\
            chevron|chinas|libya|opecs|iranian|permian|canadian|chinese|reserves|arctic|petrobras|gasoline|russias|texas\
            |oils|tullow|lng|surge|venezuelas|canadas|canada|barrels|oilfield|mexicos|irans|asian|natural|carbon|arabias|\
            norway|drillers|european|fossil|libyan|gazprom|fuels|iraqi|refiners|norways|petroleum|tankers|nigerias|libyas|\
            wti|venezuelan|emissions|repsol|statoil|brazil|mexican|electric|refinery|kurdistan|nigerian|australian|refining\
            |petrol|shows|iraqs|kuwait|syria|kurdish|kazakhstan|coal|turkey|refineries|uae|petrofac|lukoil',news.lower()):
                time=dv.find_elements_by_xpath('//*[@class="articleDetails"]/span[@class="date"]')[0].text
                time=time.lower().replace('-','').strip()
                if re.search('hour|minute|now',time):
                    time=datetime.now()
                else:
                    time=datetime.strptime(time, "%b %d, %Y")
                if len(news)!=0:
                    try:
                        writer.writerow([time.strftime("%d/%m/%Y"),news.lower().encode('utf8','ignore').decode('utf-8', 'replace')])
                    except UnicodeEncodeError as ue:
                        print('Error:'+str(ue)+'|Link:'+weblink+'Tag:'+str(news))
#         browser.implicitly_wait(np.random.randint(5,20))
        print('Scrapping Completed for '+str(PageNo))
        PageNo=PageNo-1
browser.close()
f.close()

Scrapping Completed for 1865
Scrapping Completed for 1864
Scrapping Completed for 1863
Scrapping Completed for 1862
Scrapping Completed for 1861
Scrapping Completed for 1860
Scrapping Completed for 1859
Scrapping Completed for 1858
Scrapping Completed for 1857
Scrapping Completed for 1856
Scrapping Completed for 1855
Scrapping Completed for 1854
Scrapping Completed for 1853
Scrapping Completed for 1852
Scrapping Completed for 1851
Scrapping Completed for 1850
Scrapping Completed for 1849
Scrapping Completed for 1848
Scrapping Completed for 1847
Scrapping Completed for 1846
Scrapping Completed for 1845
Scrapping Completed for 1844
Scrapping Completed for 1843
Scrapping Completed for 1842
Scrapping Completed for 1841
Scrapping Completed for 1840
Scrapping Completed for 1839
Scrapping Completed for 1838
Scrapping Completed for 1837
Scrapping Completed for 1836
Scrapping Completed for 1835
Scrapping Completed for 1834
Scrapping Completed for 1833
Scrapping Completed for 1832
Scrapping Comp

Scrapping Completed for 1582
Scrapping Completed for 1581
Scrapping Completed for 1580
Scrapping Completed for 1579
Scrapping Completed for 1578
Scrapping Completed for 1577
Scrapping Completed for 1576
Scrapping Completed for 1575
Scrapping Completed for 1574
Scrapping Completed for 1573
Scrapping Completed for 1572
Scrapping Completed for 1571
Scrapping Completed for 1570
Scrapping Completed for 1569
Scrapping Completed for 1568
Scrapping Completed for 1567
Scrapping Completed for 1566
Scrapping Completed for 1565
Scrapping Completed for 1564
Scrapping Completed for 1563
Scrapping Completed for 1562
Scrapping Completed for 1561
Scrapping Completed for 1560
Scrapping Completed for 1559
Scrapping Completed for 1558
Scrapping Completed for 1557
Scrapping Completed for 1556
Scrapping Completed for 1555
Scrapping Completed for 1554
Scrapping Completed for 1553
Scrapping Completed for 1552
Scrapping Completed for 1551
Scrapping Completed for 1550
Scrapping Completed for 1549
Scrapping Comp

Scrapping Completed for 1299
Scrapping Completed for 1298
Scrapping Completed for 1297
Scrapping Completed for 1296
Scrapping Completed for 1295
Scrapping Completed for 1294
Scrapping Completed for 1293
Scrapping Completed for 1292
Scrapping Completed for 1291
Scrapping Completed for 1290
Scrapping Completed for 1289
Scrapping Completed for 1288
Scrapping Completed for 1287
Scrapping Completed for 1286
Scrapping Completed for 1285
Scrapping Completed for 1284
Scrapping Completed for 1283
Scrapping Completed for 1282
Scrapping Completed for 1281
Scrapping Completed for 1280
Scrapping Completed for 1279
Scrapping Completed for 1278
Scrapping Completed for 1277
Scrapping Completed for 1276
Scrapping Completed for 1275
Scrapping Completed for 1274
Scrapping Completed for 1273
Scrapping Completed for 1272
Scrapping Completed for 1271
Scrapping Completed for 1270
Scrapping Completed for 1269
Scrapping Completed for 1268
Scrapping Completed for 1267
Scrapping Completed for 1266
Scrapping Comp

Scrapping Completed for 1016
Scrapping Completed for 1015
Scrapping Completed for 1014
Scrapping Completed for 1013
Scrapping Completed for 1012
Scrapping Completed for 1011
Scrapping Completed for 1010
Scrapping Completed for 1009
Scrapping Completed for 1008
Scrapping Completed for 1007
Scrapping Completed for 1006
Scrapping Completed for 1005
Scrapping Completed for 1004
Scrapping Completed for 1003
Scrapping Completed for 1002
Scrapping Completed for 1001
Scrapping Completed for 1000
Scrapping Completed for 999
Scrapping Completed for 998
Scrapping Completed for 997
Scrapping Completed for 996
Scrapping Completed for 995
Scrapping Completed for 994
Scrapping Completed for 993
Scrapping Completed for 992
Scrapping Completed for 991
Scrapping Completed for 990
Scrapping Completed for 989
Scrapping Completed for 988
Scrapping Completed for 987
Scrapping Completed for 986
Scrapping Completed for 985
Scrapping Completed for 984
Scrapping Completed for 983
Scrapping Completed for 982
Scr

Scrapping Completed for 724
Scrapping Completed for 723
Scrapping Completed for 722
Scrapping Completed for 721
Scrapping Completed for 720
Scrapping Completed for 719
Scrapping Completed for 718
Scrapping Completed for 717
Scrapping Completed for 716
Scrapping Completed for 715
Scrapping Completed for 714
Scrapping Completed for 713
Scrapping Completed for 712
Scrapping Completed for 711
Scrapping Completed for 710
Scrapping Completed for 709
Scrapping Completed for 708
Scrapping Completed for 707
Scrapping Completed for 706
Scrapping Completed for 705
Scrapping Completed for 704
Scrapping Completed for 703
Scrapping Completed for 702
Scrapping Completed for 701
Scrapping Completed for 700
Scrapping Completed for 699
Scrapping Completed for 698
Scrapping Completed for 697
Scrapping Completed for 696
Scrapping Completed for 695
Scrapping Completed for 694
Scrapping Completed for 693
Scrapping Completed for 692
Scrapping Completed for 691
Scrapping Completed for 690
Scrapping Completed 

Scrapping Completed for 438
Scrapping Completed for 437
Scrapping Completed for 436
Scrapping Completed for 435
Scrapping Completed for 434
Scrapping Completed for 433
Scrapping Completed for 432
Scrapping Completed for 431
Scrapping Completed for 430
Scrapping Completed for 429
Scrapping Completed for 428
Scrapping Completed for 427
Scrapping Completed for 426
Scrapping Completed for 425
Scrapping Completed for 424
Scrapping Completed for 423
Scrapping Completed for 422
Scrapping Completed for 421
Scrapping Completed for 420
Scrapping Completed for 419
Scrapping Completed for 418
Scrapping Completed for 417
Scrapping Completed for 416
Scrapping Completed for 415
Scrapping Completed for 414
Scrapping Completed for 413
Scrapping Completed for 412
Scrapping Completed for 411
Scrapping Completed for 410
Scrapping Completed for 409
Scrapping Completed for 408
Scrapping Completed for 407
Scrapping Completed for 406
Scrapping Completed for 405
Scrapping Completed for 404
Scrapping Completed 

Scrapping Completed for 145
Scrapping Completed for 144
Scrapping Completed for 143
Scrapping Completed for 142
Scrapping Completed for 141
Scrapping Completed for 140
Scrapping Completed for 139
Scrapping Completed for 138
Scrapping Completed for 137
Scrapping Completed for 136
Scrapping Completed for 135
Scrapping Completed for 134
Scrapping Completed for 133
Scrapping Completed for 132
Scrapping Completed for 131
Scrapping Completed for 130
Scrapping Completed for 129
Scrapping Completed for 128
Scrapping Completed for 127
Scrapping Completed for 126
Scrapping Completed for 125
Scrapping Completed for 124
Scrapping Completed for 123
Scrapping Completed for 122
Scrapping Completed for 121
Scrapping Completed for 120
Scrapping Completed for 119
Scrapping Completed for 118
Scrapping Completed for 117
Scrapping Completed for 116
Scrapping Completed for 115
Scrapping Completed for 114
Scrapping Completed for 113
Scrapping Completed for 112
Scrapping Completed for 111
Scrapping Completed 

In [4]:
binman::list_versions("chromedriver")

SyntaxError: invalid syntax (<ipython-input-4-26d6b7a742ea>, line 1)

In [2]:
scrape_moneycontrol()